*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  07.05.2024

--- 

# Detector Emission Calculation

<!--Notebook description and usage information-->
HBEFA provides different emission factors for different traffic situations e.g. Freeflow, Saturated or Stop&Go. These traffic situations are characterized by different average speeds of the vehicles, which is also provided by the traffic detectors.


In [27]:
import sys

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import time

sys.path.append('../utils')
import data_paths
from hbefa_hot_emissions import HbefaHotEmissions

# Notebook Settings

In [28]:
# year of investigation
year = 2019

# Define VISUM filename
visum_filename = "visum_links.GPKG"

#Define Counting Data filename
cnt_data_filename  = 'counting_data_combined.parquet'

# Define vehicle classes and components
vehicle_classes = ['PC', 'LCV', 'HGV', 'BUS', 'MOT']
components = ['CO2(rep)', 'CO2(total)', 'NOx', 'CO']

###
#
# Save Data as parquet file
#
##

save_results = True
save_filepath = data_paths.INVENTORY_FOLDER_PATH + 'DetectorEmissions_2019_vc_estimate.feather'

# Notebook Functions

In [29]:
def calculate_emissions(year: int,
                        vehicle_class: str,
                        vehicle_volume: int,
                        TraSit: str,
                        hbefa_gradient: str,
                        component: str, 
                        hbefa_class: HbefaHotEmissions) -> float:
    """Calculates the emissions for a single vehicle class based on the given traffic volume and hbefa Traffic Situation

    Args:
        year (int): Year of investigation
        vehicle_class (str): Vehicle Class
        vehicle_volume (int): Traffic volume of the respective vehicle class
        TraSit (str): Traffic situation 
        hbefa_gradient (str): Road gradient
        component (str): Emission component
        hbefa_class (HbefaHotEmissions): pre-initilized HBEFA object

    Returns:
        float: Emission estimate for the given input parameters
    """
    
    try:
        ef = hbefa_class.ef_dict[vehicle_class]['EFA_weighted']\
            [year, TraSit, hbefa_gradient, component]
    except KeyError:
        ef = hbefa_class.ef_dict[vehicle_class]['EFA_weighted']\
            [year, TraSit, '0%', component]
    return vehicle_volume * ef


## Import Data

In [30]:
# import speed and volume data from traffic counting stations
visum_links = gpd.read_file(data_paths.VISUM_FOLDER_PATH + visum_filename)
# raw speed and counting data from LHM counting stations
cnt_data = pd.read_parquet(data_paths.COUNTING_PATH + cnt_data_filename)

# subselect counting data for links that are in the visum network
cnt_data = cnt_data[cnt_data['road_link_id'].isin(visum_links['road_link_id'].unique())]
cnt_data = cnt_data[cnt_data['date'].between(f'{year}-01-01', f'{year}-12-31')].copy() # reduce to 2019 data

# import hbefa emission module
hbefa = HbefaHotEmissions()

Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_PC.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_LCV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_HGV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_Coach.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_MOT.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/ef_aggregated_los/EFA_HOT_Vehcat

In [32]:
road_info = pd.merge(cnt_data['road_link_id'].drop_duplicates(), visum_links[['road_type', 'hbefa_gradient',
                                       'hbefa_speed', 'speed', 'road_link_id', 'hour_capacity']], 
                    left_on = 'road_link_id', 
                    right_on = 'road_link_id', 
                    how = 'inner')

# set duplicates in road gradient to 0%
road_info = road_info.groupby('road_link_id').agg({'speed': 'first', 
                                                'hbefa_speed': 'first',
                                                'hour_capacity': 'sum',
                                                'hbefa_gradient': lambda x: x.iloc[0] if len(x)==1 else '0%'}).reset_index()
road_info.head()

,road_link_id,speed,hbefa_speed,hour_capacity,hbefa_gradient
0,38.0,60,60,7020,0%
1,60.0,90,90,3960,0%
2,72.0,60,60,14400,0%
3,419.0,50,50,4800,0%
4,810.0,50,50,1200,6%


# Prepare speed and volume dataframe

In [39]:
_cnt = cnt_data.melt(id_vars = ['date', 'road_link_id','road_type', 'vehicle_class'],
                     value_vars = [str(x) for x in range(0,24)])

_cnt['timestamp'] = _cnt.apply(lambda row: pd.Timestamp.combine(row['date'],
                                                                time(int(row['variable']))), axis =1)

_cnt = _cnt[_cnt['vehicle_class']!='SUM'] # delete 'SUM' vehicle class

# prepare volume dataset
_cnt_volume = _cnt.pivot(index = ['road_link_id', 'road_type', 'timestamp'],
                                columns = 'vehicle_class',
                                values = 'value')

_cnt_volume['SUM_PCU'] = _cnt_volume.mul(pd.Series(hbefa.car_unit_factors)).sum(axis = 1)
_cnt_volume = _cnt_volume.dropna().reset_index()

cnt_volume = pd.merge(_cnt_volume, road_info, on = 'road_link_id', how = 'inner')


# Calculate Traffic Situation by speed distribution

In [41]:
cnt_volume['TraSit']= cnt_volume.apply(lambda row: hbefa.calc_los_class(hbefa_speed=row['hbefa_speed'],
                                                                        hour_capacity=row['hour_capacity'],
                                                                        htv_car_unit = row['SUM_PCU'],
                                                                        road_type = row['road_type']),
                                       axis = 1)
                                        


# Calculate emissions for each detector


In [42]:
for c in components: 
    for vc in vehicle_classes:
        cnt_volume[f'{vc}_{c}'] = cnt_volume.apply(lambda row: calculate_emissions(year = year,
                                                                                   vehicle_class=vc,
                                                                                   vehicle_volume= row[vc],
                                                                                   TraSit=row['TraSit'],
                                                                                   hbefa_gradient=row['hbefa_gradient'],
                                                                                   component=c,
                                                                                   hbefa_class=hbefa), axis = 1)
cnt_volume.head()

,road_link_id,road_type,timestamp,BUS,HGV,LCV,MOT,PC,SUM_PCU,speed,...,PC_NOx,LCV_NOx,HGV_NOx,BUS_NOx,MOT_NOx,PC_CO,LCV_CO,HGV_CO,BUS_CO,MOT_CO
0,38.0,TrunkRoad/Primary-City,2019-01-05 00:00:00,7.0,37.0,64.0,0.0,970.0,1138.75,60,...,323.967821,52.447964,66.147866,22.507666,0.000000,579.913599,20.411354,30.791973,11.521091,0.000000
1,38.0,TrunkRoad/Primary-City,2019-01-05 01:00:00,3.0,42.0,55.0,2.0,536.0,703.25,60,...,179.017270,45.072469,75.086767,9.646143,0.177144,320.447102,17.541007,34.953050,4.937610,7.414128
2,38.0,TrunkRoad/Primary-City,2019-01-05 02:00:00,1.0,35.0,52.0,1.0,333.0,475.25,60,...,111.217819,42.613971,62.572306,3.215381,0.088572,199.083741,16.584225,29.127542,1.645870,3.707064
3,38.0,TrunkRoad/Primary-City,2019-01-05 03:00:00,1.0,29.0,36.0,8.0,296.0,414.25,60,...,98.860284,29.501980,51.845625,3.215381,0.708578,176.963325,11.481387,24.134249,1.645870,29.656513
4,38.0,TrunkRoad/Primary-City,2019-01-05 04:00:00,2.0,35.0,37.0,3.0,400.0,531.00,60,...,133.594978,30.321479,62.572306,6.430762,0.265717,239.139628,11.800314,29.127542,3.291740,11.121192


## Save Results

In [43]:
if save_results: 
    cnt_volume.to_feather(save_filepath)